# PyTorch Tensor Cheat Sheet

## 改变形状

* `torch.reshape(input, shape)`：改变形状，可以有一个维度是-1，会 **尽可能** 返回原始数据的一个视图（view）而不进行数据拷贝
* `torch.Tensor.view(*shape)`：改变形状，可以有一个维度是-1，会返回原始数据的一个视图（即与原tensor共享数据），若不能则报错
* `torch.squeeze(input, dim=None)`：把 **所有** 大小为1的维度移除，当`dim`指定时，只移除该维度（大小必须为1）
* `torch.unsqueeze(input, dim)`：在`dim`处插入一个大小为1的维度

## 交换维度（转置）

* `torch.Tensor.transpose(dim0, dim1)`：交换两个维度，返回原始tensor的视图
* `torch.Tensor.permute(*dims)`：按照`*dims`指定的顺序对维度进行排序，返回原始tensor的视图

## 复制维度

* `torch.Tensor.expand(*sizes)`：将tensor的形状广播为`*sizes`，返回一个视图
    * 不想改动的维度可设置为-1
    * 可在开头插入新维度，但不能删除维度，且插入的新维度不能设为-1
    * 扩展后的tensor中或许有多个元素共享存储空间，这可能导致错误，所以如果要对它写，先进行复制
* `torch.Tensor.repeat(*sizes)`：行为与`expand`相同，但是会拷贝数据
* `torch.repeat_interleave(input, repeats, dim=None) `：不常用，自行查阅

## 复制

TODO

## 拼接

TODO

## 拆分

TODO

## 广播机制

PyTorch和Numpy中的许多针对tensor或者ndarray的操作都要求两者的形状相同，但是有的时候我们想有种机制**自动地去复制某些维度**，以在不通过复制数据的情况下让tensor的形状相匹配，这种机制就叫做**广播**。

许多PyTorch操作支持Numpy的[广播语义](https://numpy.org/doc/stable/user/basics.broadcasting.html)。

广播就是对于两个形状不同的tensor，在概念上把互相不匹配的维度自动地进行扩展，使得两个tensor的形状相同，避免不必要的复制。

上面的定义暗示了不是任何两个tensor都可以进行广播，它们的形状必须是兼容的，或者说**可广播的**（boradcastable）。

当满足下面条件时，两个tensor是可广播的：

* 每个tensor都至少有1个维度（及形状都不能是`torch.Size([])`）
* 从最后一个维度开始往前进行比较，两个维度必须是相等的，或者其中一个为1，或者其中一个不存在

可广播的两个tensor，在从后往前比较各维度的过程中有三种情况：

* 如果两个维度相等，则什么都不做
* 如果其中一个为1，另一个为k，则把为1的那个tensor的这个维度复制k次，使得两个维度相等
* 如果其中一个不存在，另一个为k，则把不存在的那个tensor在这个位置增加一个维度，把该维度的数据复制k次，使得两个维度相等

注意，维度不存在情况只能出现在维度较少的tensor的开头。

下面是一些例子：

In [1]:
import torch

a = torch.randn(5, 3, 4, 1)
b = torch.randn(   3, 1, 1)
(a + b).shape  # of shape (5, 3, 4, 1)

torch.Size([5, 3, 4, 1])

In [2]:
a = torch.randn(5, 3, 1, 2)
b = torch.randn(5, 1, 4, 2)
(a + b).shape  # of shape (5, 3, 4, 1)

torch.Size([5, 3, 4, 2])

In [3]:
a = torch.randn(5, 3, 1, 2)
b = torch.randn(5, 2, 4, 2)
# (a + b).shape  # RuntimeError

 注意广播后的结果有可能不同于原来的任何一个矩阵，而in-place操作（例如`Tensor.add_`等末尾带`_`的函数，或者它的等价形式`a += b`）不允许改变原有tensor的形状，这样的话一些in-place的操作有可能触发`RuntimeError`

In [4]:
a = torch.randn(3, 1, 4)
b = torch.randn(3, 5, 1)
# a += b  # RuntimeError

在老版本的PyTorch中，一些逐点（pointwise）操作只要求两个tensor的元素个数一样，然后把两个tensor视作一维后逐点操作，而在引入广播语义后，这些操作会对两个tensor做广播，这可能导致一些向后兼容性问题。

In [5]:
a = torch.randn(4, 1)
b = torch.randn(   4)
(a + b).shape  # result is of shape(4, 1) in prior versions, of shape(4, 4) in the current version

torch.Size([4, 4])

## 加减乘除

这些操作都是二元操作，并且一般可以通过指定一个`out`参数来指示结果存放的tensor。

### 加法

* `a + b`相当于`torch.add(a, b)`
* `torch.add`：逐点相加，支持广播
* `torch.sum`：默认返回所有元素之和
    * 若指定`dim`参数，则沿着该维度求和
    * `dim`可以是一个列表
    * 求和后`dim`指示的维度消失，如果不想让它消失，则指定`keepdim=True`。

### 减法

* `a - b`相当于`torch.sub(a, b)`
* `torch.sub`：逐点减法，支持广播
* `torch.subtract`：等价于`torch.sub`

### 乘法

我们最常用的操作就是乘法，各类乘法也是PyTorch中最复杂、最令人困惑的操作。

最常用的是`torch.bmm`和`torch.matmul`，都用来进行矩阵乘法。

* `a * b`相当于`torch.mul(a, b)`
* `a @ b`相当于`torch.matmul(a, b)`

* `torch.mul`：逐点乘积，或者叫哈达玛积（Hadamard product），支持广播
* `torch.multiply`：相当于`torch.mul`
* `torch.mm`：2维tensor的矩阵乘法，不支持广播
* `torch.bmm`：3维tensor的批量矩阵乘法，第一个维度是batch size，不支持广播
* `torch.matmul`：通用的矩阵乘法，支持广播 *（最灵活，使用时检查参数维度，以免出bug）*
    * 如果两个tensor都是1维的，进行点积（相当于`torch.dot`）
        * 参数形状为`(n,)`和`(n,)`，结果形状为`()`
    * 如果两个tensor都是2维的，进行矩阵乘法（相当于`torch.mm`）
        * 参数形状为`(m, n)`和`(n, p)`，结果形状为`(m, p)`
    * 如果第一个tensor是1维的，第二个tensor是二维的，等于行向量乘矩阵
        * 参数形状为`(m,)`和`(m, n)`，结果形状为`(n,)`
    * 如果第一个tensor是2维的，第二个tensor是一维的，等于矩阵乘列向量
        * 参数形状为`(m, n)`和`(n,)`，结果形状为`(m,)`
    * 如果两个tensor都至少1维，且至少有一个tensor至少是3维的，对batch维度进行广播后进行矩阵乘法
        * 两个tensor的后两个维度必须满足矩阵乘法规则，然后对前面的维度进行广播
        * 参数形状为`(*batch, m, n)`和`(*batch, n, p)`，结果形状为`(*batch, n, p)`
* `torch.dot`：向量点积（dot product），或叫数量积（scalar product），两个tensor必须都是1维的且长度相同
* `torch.outer`：向量外积（outer product），两个tensor必须都是1维的，结果是一个矩阵
    * 参数形状为`(m,)`和`(n,)`，结果形状为`(m, n)`，结果的`torch.outer(a, b)[i, j] == a[i] * b[j]`
* `torch.cross`：向量叉积（cross product），有时候也叫外积（exterior product），两个tensor必须相同形状且有一个维度是3，不支持广播
    * 参数和结果形状都是`(*dim1, 3, *dim2)`
    * 可以指定维度，如果不指定，会选择最开头的为3的维度，最好指定以避免bug

### 除法

* `a / b`相当于`torch.div(a, b)`
* `a // b`相当于`torch.floor_divide(a, b)`
* `torch.div`：浮点数除法，目前版本不支持两个整数tensor的除法，对整数tensor可以用`//`或者`torch.floor_divide`
* `torch.true_divide`：等价于`torch.div`
* `torch.floor_divide`：**向零舍入** ，如果两个参数都是整数，则结果为整数，否则为浮点数

## gather和scatter

TODO